In [1]:
pip install -q langchain faiss-cpu requests numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 48.1 MB/s eta 0:00:00


In [2]:
pip install -q langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [10]:
pip install -q euriai[langchain]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 1.5 MB/s eta 0:00:00


In [ ]:
EURI_API_KEY= "euri-....."

In [3]:
import requests
import numpy as np
from langchain.embeddings.base import Embeddings

class EuriaiEmbedding(Embeddings):
    def __init__(self, api_key: str, model: str = "text-embedding-3-small"):
        self.api_key = api_key
        self.model = model

    def embed_documents(self, texts):
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text):
        url = "https://api.euron.one/api/v1/euri/embeddings"
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }
        payload = {
            "input": text,
            "model": self.model
        }
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        embedding = np.array(response.json()["data"][0]["embedding"]).tolist()
        return embedding


In [4]:
from langchain.vectorstores import FAISS
from langchain.schema import Document

# Sample documents
documents = [
    Document(page_content="EURI provides AI APIs for developers."),
    Document(page_content="LangChain integrates LLMs with tools, memory, and agents."),
    Document(page_content="The weather is sunny today."),
    Document(page_content="FAISS is used for vector similarity search."),
    Document(page_content="EURI embedding API returns dense vector representations.")
]

# Initialize EURI embedding model
euriai_embedder = EuriaiEmbedding(api_key= EURI_API_KEY)

# Build FAISS index
vectorstore = FAISS.from_documents(documents, euriai_embedder)
retriever = vectorstore.as_retriever(k=3)

In [5]:
query = "What is the use of EURI in AI?"
retrieved_docs = retriever.invoke(query)

In [6]:
retrieved_docs

[Document(id='11b5d4af-27ea-4c5e-b207-f05adfe053ab', metadata={}, page_content='EURI provides AI APIs for developers.'),
 Document(id='4573e13f-5437-4c3c-a4c7-5b36f793b4d0', metadata={}, page_content='EURI embedding API returns dense vector representations.'),
 Document(id='690b1ac2-b227-4907-88b7-615c8952bab1', metadata={}, page_content='FAISS is used for vector similarity search.'),
 Document(id='c50b0ac5-dbb9-4be9-80d2-2ed2c0d3f5ea', metadata={}, page_content='LangChain integrates LLMs with tools, memory, and agents.')]

In [11]:
from euriai.langchain import create_chat_model

chat_model = create_chat_model(
    api_key= EURI_API_KEY,
    model="gpt-4.1-nano",
    temperature=0.2
)

def rerank_with_euri(query, docs):
    scored = []
    for doc in docs:
        prompt = (
            f"Rate relevance (0-10) for this document:\n\n"
            f"Query: {query}\n"
            f"Document: {doc.page_content}\n"
            f"Answer only with a number:"
        )
        response = chat_model.invoke(prompt)
        try:
            score = float(response.content.strip().split()[0])
        except:
            score = 0.0
        scored.append((doc, score))
    reranked = sorted(scored, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked]

In [12]:
reranked_docs = rerank_with_euri(query, retrieved_docs)


In [13]:
reranked_docs

[Document(id='11b5d4af-27ea-4c5e-b207-f05adfe053ab', metadata={}, page_content='EURI provides AI APIs for developers.'),
 Document(id='4573e13f-5437-4c3c-a4c7-5b36f793b4d0', metadata={}, page_content='EURI embedding API returns dense vector representations.'),
 Document(id='690b1ac2-b227-4907-88b7-615c8952bab1', metadata={}, page_content='FAISS is used for vector similarity search.'),
 Document(id='c50b0ac5-dbb9-4be9-80d2-2ed2c0d3f5ea', metadata={}, page_content='LangChain integrates LLMs with tools, memory, and agents.')]

In [14]:
# Create context
context = "\n".join([doc.page_content for doc in reranked_docs])

# Ask LLM to answer
final_prompt = f"""You are an expert assistant.
Context:
{context}

Question: {query}
Answer:"""

response = chat_model.invoke(final_prompt)
print("\n💬 Final Answer:")
print(response.content)


💬 Final Answer:
EURI provides AI APIs that enable developers to incorporate advanced functionalities such as generating dense vector embeddings, which are useful for tasks like similarity search, clustering, and retrieval. Specifically, the EURI embedding API transforms textual or other data into high-dimensional vector representations that can be efficiently stored and searched using tools like FAISS. Additionally, when integrated with frameworks like LangChain, EURI's APIs facilitate building sophisticated AI applications that leverage large language models alongside vector-based retrieval and memory components.
